In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from keras import Sequential, layers, Input, regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from tensorflow import keras

from scripts.data_cleaner import filter_top_cpv_categories
from scripts.preprocess_pipeline import create_pipeline


2025-06-11 12:08:46.196296: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 12:08:46.196940: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 12:08:46.199638: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 12:08:46.206983: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749636526.219863   58336 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749636526.22

In [2]:
DATA_PATH = os.path.join(os.path.dirname(os.getcwd()),
                        'data')
MODEL_PATH = os.path.join(os.path.dirname(os.getcwd()),
                            'models', 'estimation')
os.makedirs(MODEL_PATH, exist_ok=True)


df = pd.read_csv(os.path.join(DATA_PATH,'data_clean.csv'))

/tmp/ipykernel_58336/1184559175.py:8: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(DATA_PATH,'data_clean.csv'))


In [3]:
df = filter_top_cpv_categories(df, top_n=150, cpv_column='codeCPV_3')
df.drop(df[df['montant'] > 999_999].index, inplace=True)

y = np.log1p(df['montant'])


bins = np.linspace(y.min(), y.max(), 101)
#bins = np.delete(bins, 1)

#bins = bins[2:]

# Créer les étiquettes correspondantes
labels = list(range(0, len(bins)-1))

# Ajouter la colonne fourchette_de_prix
df['bins'] = pd.cut(y, bins=bins, labels=labels, include_lowest=True, right=True)

df['bins'] = df['bins'].astype('int')
df = df.drop(columns='montant')

X = df.drop(columns=['bins', 'titulaire_tranche_effectif', 'titulaire_categorie'])
y = df['bins']

df[['acheteur_tranche_effectif', 'acheteur_categorie']] = df[['acheteur_tranche_effectif', 'acheteur_categorie']].fillna('null')


Filtered from 392 to 150 CPV categories, keeping 279174 rows out of 286850


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=X['codeCPV_3'])

numerical_columns = ['dureeMois', 'offresRecues', 'annee']

binary_columns = ['sousTraitanceDeclaree', 'origineFrance',
                          'marcheInnovant', 'idAccordCadre']

categorical_columns = ['procedure', 'nature', 'formePrix', 'ccag',
                               'typeGroupementOperateurs', 'tauxAvance_cat',
                               'codeCPV_3', 'acheteur_tranche_effectif', 'acheteur_categorie']

pipeline = create_pipeline(numerical_columns, binary_columns, categorical_columns)

X_train_preproc = pipeline.fit_transform(X_train)

y_train_cat = to_categorical(y_train)

In [5]:
y_train_cat.shape

(190530, 100)

In [6]:
from tensorflow import keras
print(keras.__version__)

ImportError: Keras cannot be imported. Check that it is installed.

In [ ]:
model_name = 'model_montant_100.keras'

model = keras.models.load_model(os.path.join(MODEL_PATH,model_name))  # or 'my_model' if it's a directory

model.summary()

In [ ]:
X_train_preproc = pipeline.fit_transform(X_train)

X_test_preproc = pipeline.transform(X_test)
y_test_cat = to_categorical(y_test)

model.evaluate(X_test_preproc, y_test_cat, verbose=1)

In [ ]:
y_pred =model.predict(X_test_preproc)